# 05d - Vertex AI > Training > Training Pipelines - With Python file

### 05 Series Overview
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the `05` notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  Training Pipeline that trains and saves models from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  Hyperparameter Tuning Jobs from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

### This Notebook (`05d`): An extension of `05a` as a Training Pipeline that saves the model to Vertex AI > Models 
This notebook trains the same Tensorflow Keras model from `05` by first modifying and saving the training code to a python script (same as `05a`).  The script is then used as an input for a Vertex AI > Training > Custom Training Job that is assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service.  This is done with the [Vertex AI Python SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#) using the class [`aiplatform.CustomJob.CustomTrainingJob()`](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#google.cloud.aiplatform.CustomTrainingJob).

The client used here is `aiplatform.CustomTrainingJob.(script_path = )` which is very similar to what was used for a script in `05a` `aiplatform.CustomJob.from_local_script(script_path = )`. The functional difference is that this method automatically uploads the final saved model to Vertex AI > Models.  Running the job this way first triggers a job in Vertex AI > Training > Training Pipelines.  This Training Pipeline triggers a Custom Job in Vertex AI > Training > Custom Jobs.  If The Custom Job completes successfully then the final saved model is registered in Vertex AI > Models.

The training can be reviewed with Vertex AI's managed Tensorboard under Vertex AI > Experiments > Experiments, or by clicking on the `05d...` custom job under Vertex AI > Training > Custom Jobs and then clicking the 'Open Tensorboard' link.

<img src="architectures/overview/Training.png">

### Prerequisites:
-  01 - BigQuery - Table Data Source
-  Understanding:
    -  05 - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow
        -  Contains a more granular review of the Tensorflow model training

### Overview:
-  Setup
-  Connect to Tensorboard instance from 05
-  Create a `train.py` Python script that recreates the local training in 04a
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Custom training job with aiplatform.CustomTrainingJob
      -  Run job with .run - returns the uploaded model name
   -  Create Endpoint with Vertex AI with aiplatform.Endpoint.create
      -  Deploy model to endpoint with .deploy 
-  Online Prediction demonstrated using Vertex AI Endpoint with deployed model
   -  Get records to score from BigQuery table
   -  Prediction with aiplatform.Endpoint.predict
   -  Prediction with REST
   -  Prediction with gcloud (CLI)

### Resources:
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/05d_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/05d_console.png">

---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'fraud'
NOTEBOOK = '05d'

# Resources
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [2]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [5]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'764015827198-compute@developer.gserviceaccount.com'

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [14]:
tb = aiplatform.Tensorboard.list(filter=f'display_name={DATANAME}')
if tb:
    tb = tb[0]
else:
    tb = aiplatform.Tensorboard.create(display_name = DATANAME, labels = {'notebook':f'{DATANAME}'})

In [16]:
tb.resource_name

'projects/715288179162/locations/us-central1/tensorboards/6212451803166932992'

---
## Training

### Assemble Python File for Training

Create the main python trainer file as `/train.py`:

In [9]:
%%writefile {DIR}/train.py

# package import
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf
from google.cloud import bigquery
import argparse
import os
import sys

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--epochs', dest = 'epochs', default = 10, type = int, help = 'Number of Epochs')
parser.add_argument('--batch_size', dest = 'batch_size', default = 32, type = int, help = 'Batch Size')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str, nargs='*')
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--dataname', dest = 'dataname', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--notebook', dest = 'notebook', type=str)
args = parser.parse_args()

# built in parameters for data source:
PROJECT_ID = args.project_id
DATANAME = args.dataname
REGION = args.region
NOTEBOOK = args.notebook

# clients
bigquery = bigquery.Client(project = PROJECT_ID)

# get schema from bigquery source
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bigquery.query(query = f'SELECT DISTINCT {args.var_target} FROM {DATANAME}.{DATANAME}_prepped WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]

# Make a list of columns to omit
OMIT = args.var_omit + ['splits']

# use schema to prepare a list of columns to read from BigQuery
selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

# all the columns in this data source are either float64 or int64
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in schema[~schema.column_name.isin(OMIT)].data_type.tolist()]

# remap input data to Tensorflow inputs of features and target
def transTable(row_dict):
    target=row_dict.pop(args.var_target)
    target = tf.one_hot(tf.cast(target,tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

# function to setup a bigquery reader with Tensorflow I/O
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

train = bq_reader('TRAIN').parallel_read_rows().prefetch(1).map(transTable).shuffle(args.batch_size*10).batch(args.batch_size)
validate = bq_reader('VALIDATE').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
test = bq_reader('TEST').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)

# Logistic Regression

# model input definitions
feature_columns = {header: tf.feature_column.numeric_column(header) for header in selected_fields if header != args.var_target}
feature_layer_inputs = {header: tf.keras.layers.Input(shape = (1,), name = header) for header in selected_fields if header != args.var_target}

# feature columns to a Dense Feature Layer
feature_layer_outputs = tf.keras.layers.DenseFeatures(feature_columns.values())(feature_layer_inputs)

# batch normalization then Dense with softmax activation to nclasses
layers = tf.keras.layers.BatchNormalization()(feature_layer_outputs)
layers = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax)(layers)

# the model
model = tf.keras.Model(
    inputs = feature_layer_inputs,
    outputs = layers
)
opt = tf.keras.optimizers.SGD() #SGD or Adam
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy', tf.keras.metrics.AUC(curve='PR')]
)

# setup tensorboard logs and train
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], histogram_freq=1)
history = model.fit(train, epochs = args.epochs, callbacks = [tensorboard_callback], validation_data = validate)

# output the model save files
model.save(os.getenv("AIP_MODEL_DIR"))

Writing temp/05c/train.py


### Setup Training Job

In [10]:
customJob = aiplatform.CustomTrainingJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    script_path = f"{DIR}/train.py",
    container_uri = TRAIN_IMAGE,
    requirements = ['tensorflow_io'],
    staging_bucket = f"{URI}/{TIMESTAMP}",
    model_serving_container_image_uri = DEPLOY_IMAGE,
    labels = {'notebook':f'{NOTEBOOK}'}
)

### Run Training Job

In [11]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--dataname=" + DATANAME,
    "--region=" + REGION,
    "--notebook=" + NOTEBOOK
]

In [12]:
model = customJob.run(
    base_output_dir = f"{URI}/{TIMESTAMP}",
    service_account = SERVICE_ACCOUNT,
    args = CMDARGS,
    replica_count = 1,
    machine_type = TRAIN_COMPUTE,
    accelerator_count = 0,
    tensorboard = tb.resource_name
)

INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://statmike-demo2/fraud/models/05c/20211202194701/aiplatform-2021-12-02-19:47:22.923-aiplatform_custom_trainer_script-0.1.tar.gz.
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://statmike-demo2/fraud/models/05c/20211202194701 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7840134732066062336?project=764015827198
INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/764015827198/locations/us-central1/trainingPipelines/7840134732066062336 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7038071785929048064?project=764015827198
INFO:google.cloud.aiplatform.training_jobs:View tensorboard:
https://us-central1.tensorboard.googleusercontent.com/

In [13]:
model.display_name

'05c_fraud_20211202194701-model'

---
## Serving

### Create An Endpoint

In [14]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/764015827198/locations/us-central1/endpoints/7105422370688466944/operations/206905067322736640
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/764015827198/locations/us-central1/endpoints/7105422370688466944
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/764015827198/locations/us-central1/endpoints/7105422370688466944')


In [15]:
endpoint.display_name

'05c_fraud_20211202194701'

### Deploy Model To Endpoint

In [16]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/764015827198/locations/us-central1/models/2661420671589941248 to Endpoint : projects/764015827198/locations/us-central1/endpoints/7105422370688466944
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/764015827198/locations/us-central1/endpoints/7105422370688466944/operations/8943888344421498880
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/764015827198/locations/us-central1/endpoints/7105422370688466944


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [17]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [18]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,79470,1.073063,-0.026793,0.796502,1.754389,-0.219910,0.930349,-0.515894,0.461495,-0.132966,...,0.048539,-0.258218,0.171861,1.028847,-0.037692,-0.013125,0.0,0,07fdced0-3837-47a1-9526-64d74ad9b113,TEST
1,122082,1.878563,0.020785,-1.621113,2.908813,2.507847,4.709442,-0.830626,1.136154,-0.395755,...,0.121098,0.707538,0.140100,0.155684,0.016375,-0.053892,0.0,0,7c1f61ba-7586-414e-ba8a-1c4385d59933,TEST
2,11494,-1.169744,0.462878,1.587579,-1.252560,1.012817,-0.918413,0.817632,-0.522804,1.388247,...,-0.042144,-0.021269,-0.283979,0.556661,-0.191722,-0.071773,0.0,0,a3046c87-cee5-40fd-9302-4d230b823246,TEST
3,58900,-2.728403,-2.340346,2.551039,2.623092,3.014728,-0.807294,-2.286621,0.757561,-1.279297,...,0.370139,-0.212784,-0.104956,1.355781,-0.053063,0.125386,0.0,0,bc3f2800-a4bb-4077-b017-f55f03c4f00c,TEST


In [19]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [20]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [21]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999908686, 9.13112235e-05]], deployed_model_id='8726753420913934336', explanations=None)

In [22]:
prediction.predictions[0]

[0.999908686, 9.13112235e-05]

In [23]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [24]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [25]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999908686,
      9.13112235e-05
    ]
  ],
  "deployedModelId": "8726753420913934336",
  "model": "projects/764015827198/locations/us-central1/models/2661420671589941248",
  "modelDisplayName": "05c_fraud_20211202194701-model"
}


### Get Predictions: gcloud (CLI)

In [26]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999908686, 9.13112235e-05]]


---
## Remove Resources
see notebook "99 - Cleanup"